In [1]:
const cudf = require('@rapidsai/cudf');

In [2]:

const regexps = [
  /Cloud|Overcast/,
  /Rain|T-Storm|Thunderstorm|Squalls|Drizzle/,
  /Snow/,
  /Fog/,
  /Ice|Hail|Freezing|Sleet/,
  /Dust|Smoke|Sand/,
];

console.log('');

const weather_condition_gpu = cudf.DataFrame.readCSV({
  header: 0,
  sourceType: 'files',
  sources: [`data/US_Accidents_Dec20.csv`],
  dataTypes: {
    id: 'str', source: 'str', tmc: 'float64', severity: 'int32', start_time: 'str', end_time: 'str',
    start_lat: 'float64', start_lng: 'float64', end_lat: 'float64', end_lng: 'float64',
    distance: 'float64', description: 'str', number: 'int32', street: 'str', side: 'str',
    city: 'str', county: 'str', state: 'str', zipcode: 'str', country: 'str', timezone: 'str', airport_code: 'str',
    weather_timestamp: 'str', temperature: 'float64', wind_chill: 'float64', humidity: 'float64', pressure: 'float64',
    visibility: 'float64', wind_direction: 'str', wind_speed: 'float64', precipitation: 'float64', weather_condition: 'str',
    amenity: 'bool', bump: 'bool', crossing: 'bool', give_way: 'bool', junction: 'bool', no_exit: 'bool', railway: 'bool',
    roundabout: 'bool', station: 'bool', stop: 'bool', traffic_calming: 'bool', traffic_signal: 'bool', turning_loop: 'bool',
    sunrise_sunset: 'str', civil_twilight: 'str', nautical_twighlight: 'str', astronomical_twighlight: 'str'
  },
}).get('weather_condition');

console.time(`GPU time`);

regexps.forEach((regexp) => {
  console.time(`${regexp.source} time`);
  const matches = weather_condition_gpu.containsRe(regexp.source).sum();
  console.timeEnd(`${regexp.source} time`);
  console.log(`${regexp.source} matches: ${matches.toLocaleString()}`);
});

console.timeEnd(`GPU time`);

console.log('');

const weather_condition_cpu = (() => {
  const categorical = weather_condition_gpu.cast(new cudf.Categorical(new cudf.Utf8String));
  const categories = [...categorical.categories];
  const codes = [...categorical.codes];
  return codes.map((i) => categories[i]);
})();

console.time(`CPU time`);

regexps.forEach((regexp) => {
  console.time(`${regexp.source} time`);
  const matches = weather_condition_cpu.reduce((matches, weather_condition) => {
    return matches + (regexp.exec(weather_condition) || []).length;
  }, 0);
  console.timeEnd(`${regexp.source} time`);
  console.log(`${regexp.source} matches: ${matches.toLocaleString()}`);
});

console.timeEnd(`CPU time`);

console.log('');

Error: cuDF failure at: /opt/rapids/node/.cache/source/cudf/0f5c9e0eb240e5571bfbd25187e94a554506376c/cpp/include/cudf/utilities/type_dispatcher.hpp:502: Unsupported type_id.